# Выборочные ACF и PACF (пакет [`sktime`](https://www.sktime.net/en/stable/get_started.html))

In [ ]:
import numpy as np
import pandas as pd

# Выборочные ACF & PACF как трансформеры
from sktime.transformations.series.acf import AutoCorrelationTransformer, PartialAutoCorrelationTransformer
# Базовые трансформеры
from sktime.transformations.series.boxcox import LogTransformer
from sktime.transformations.series.difference import Differencer
# Визуализация
from sktime.utils.plotting import plot_correlations

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

## Визуализация выборочных ACF & PACF

Визуализируем сам ряд и выборочные ACF & PACF для исходного ряда

In [ ]:
plot_correlations(y, lags=25, zero_lag=True, alpha=0.05, suptitle='Market Yield', acf_title='ACF', pacf_title='PACF')

plt.show()

Вычислим первые 5 коэффициентов ACF

In [ ]:
acf_transformer = AutoCorrelationTransformer(n_lags=5)
acf_transformer.fit_transform(y)

Вычислим первые 5 коэффициентов PACF

In [ ]:
pacf_transformer = PartialAutoCorrelationTransformer(n_lags=5)
pacf_transformer.fit_transform(y)

## Выборочные ACF & PACF как части pipeline

In [ ]:
z = web.DataReader(name='GDP', data_source='fred', start='1990-01-01')

In [ ]:
pipe_acf = LogTransformer() * Differencer(lags=1, na_handling='drop_na') * PartialAutoCorrelationTransformer(n_lags=5)
pipe_acf.fit_transform(z)

In [ ]:
pipe_pacf = LogTransformer() * Differencer(lags=1, na_handling='drop_na') * AutoCorrelationTransformer(n_lags=5)
pipe_pacf.fit_transform(z)

In [ ]:
pipe = LogTransformer() * Differencer(lags=1, na_handling='drop_na')
z_pip = pipe.fit_transform(z)

plot_correlations(z_pip, lags=25, zero_lag=True, alpha=0.05, suptitle='GDP', acf_title='ACF', pacf_title='PACF')

plt.show()